In [1]:
# issue_qwen2.5_coder_7B_FIM_bank_line2
# 250319

In [2]:
import os
if '/usr/local/cuda' not in os.environ['LD_LIBRARY_PATH']:
    os.environ['LD_LIBRARY_PATH'] = '/opt/miniforge3/lib/python3.12/site-packages/nvidia/nvjitlink/lib:' + os.environ['LD_LIBRARY_PATH'] + ':/usr/local/cuda/lib64'
os.environ["CUDA_VISIBLE_DEVICES"] = "4" 
from transformers import AutoTokenizer, AutoModelForCausalLM
device = "cuda" # the device to load the model onto
model_name_or_path = '/root/workspace/models/Qwen2.5-Coder-7B'
TOKENIZER = AutoTokenizer.from_pretrained(model_name_or_path)
MODEL = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", torch_dtype='bfloat16', attn_implementation="flash_attention_2").eval()
# hm_fim中的示例: ans_score_blankline.jsonl第2行(第1开始计数) 
# ans_score_blankline.jsonl生成方法：改造后的humaneval_fim.py（增加了输出字段）。
# 原始humaneval_fim.py：https://github.com/QwenLM/Qwen2.5-Coder/blob/main/qwencoder-eval/base/benchmarks/fim-bench/hm_fim/humaneval_fim.py
input_text = '''<|fim_prefix|>from typing import List

def triples_sum_to_zero(l: List[int]) -> bool:
    """
    triples_sum_to_zero takes a list of integers as an input.
    it returns True if there are three distinct elements in the list that
    sum to zero, and False otherwise.

    >>> triples_sum_to_zero([1, 3, 5, 0])
    False
    >>> triples_sum_to_zero([1, 3, -2, 1])
    True
    >>> triples_sum_to_zero([1, 2, 3, 7])
    False
    >>> triples_sum_to_zero([2, 4, -5, 3, 9, 7])
    True
    >>> triples_sum_to_zero([1])
    False
    """
<|fim_suffix|>    l.sort()
    for i in range(len(l)):
        for j in range(i + 1, len(l)):
            for k in range(j + 1, len(l)):
                if l[i] + l[j] + l[k] == 0:
                    return True
    return False
<|fim_middle|>'''
model_inputs = TOKENIZER([input_text], return_tensors="pt").to(device)
generated_ids = MODEL.generate(model_inputs.input_ids, max_new_tokens=512, do_sample=False)[0]
generated_ids.shape
output_text = TOKENIZER.decode(generated_ids[len(model_inputs.input_ids[0]):], skip_special_tokens=False)
print(f"Prompt:\n{input_text}\n\nGenerated text:\n{output_text}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Prompt:
<|fim_prefix|>from typing import List

def triples_sum_to_zero(l: List[int]) -> bool:
    """
    triples_sum_to_zero takes a list of integers as an input.
    it returns True if there are three distinct elements in the list that
    sum to zero, and False otherwise.

    >>> triples_sum_to_zero([1, 3, 5, 0])
    False
    >>> triples_sum_to_zero([1, 3, -2, 1])
    True
    >>> triples_sum_to_zero([1, 2, 3, 7])
    False
    >>> triples_sum_to_zero([2, 4, -5, 3, 9, 7])
    True
    >>> triples_sum_to_zero([1])
    False
    """
<|fim_suffix|>    l.sort()
    for i in range(len(l)):
        for j in range(i + 1, len(l)):
            for k in range(j + 1, len(l)):
                if l[i] + l[j] + l[k] == 0:
                    return True
    return False
<|fim_middle|>

Generated text:
<|endoftext|>


In [7]:
d = {"name": "humaneval-HumanEval_40_triples_sum_to_zero.json-L20", "language": "py", "prompt": "from typing import List\n\ndef triples_sum_to_zero(l: List[int]) -> bool:\n    \"\"\"\n    triples_sum_to_zero takes a list of integers as an input.\n    it returns True if there are three distinct elements in the list that\n    sum to zero, and False otherwise.\n\n    >>> triples_sum_to_zero([1, 3, 5, 0])\n    False\n    >>> triples_sum_to_zero([1, 3, -2, 1])\n    True\n    >>> triples_sum_to_zero([1, 2, 3, 7])\n    False\n    >>> triples_sum_to_zero([2, 4, -5, 3, 9, 7])\n    True\n    >>> triples_sum_to_zero([1])\n    False\n    \"\"\"\n", "suffix": "    l.sort()\n    for i in range(len(l)):\n        for j in range(i + 1, len(l)):\n            for k in range(j + 1, len(l)):\n                if l[i] + l[j] + l[k] == 0:\n                    return True\n    return False\n", "canonical_solution": "    assert len(l) > 0, \"The input list cannot be empty\"", "tests": "def check(candidate):\n    assert candidate([1, 3, 5, 0]) == False\n    assert candidate([1, 3, 5, -1]) == False\n    assert candidate([1, 3, -2, 1]) == True\n    assert candidate([1, 2, 3, 7]) == False\n    assert candidate([1, 2, 5, 7]) == False\n    assert candidate([2, 4, -5, 3, 9, 7]) == True\n    assert candidate([1]) == False\n    assert candidate([1, 3, 5, -100]) == False\n    assert candidate([100, 3, 5, -100]) == False\n\ndef test_check():\n    check(triples_sum_to_zero)\n\ntest_check()\n", "input": "<|fim_prefix|>from typing import List\n\ndef triples_sum_to_zero(l: List[int]) -> bool:\n    \"\"\"\n    triples_sum_to_zero takes a list of integers as an input.\n    it returns True if there are three distinct elements in the list that\n    sum to zero, and False otherwise.\n\n    >>> triples_sum_to_zero([1, 3, 5, 0])\n    False\n    >>> triples_sum_to_zero([1, 3, -2, 1])\n    True\n    >>> triples_sum_to_zero([1, 2, 3, 7])\n    False\n    >>> triples_sum_to_zero([2, 4, -5, 3, 9, 7])\n    True\n    >>> triples_sum_to_zero([1])\n    False\n    \"\"\"\n<|fim_suffix|>    l.sort()\n    for i in range(len(l)):\n        for j in range(i + 1, len(l)):\n            for k in range(j + 1, len(l)):\n                if l[i] + l[j] + l[k] == 0:\n                    return True\n    return False\n<|fim_middle|>", "model_gen": "", "model_gen_token_ids": [151643], "model_gen2": "<|endoftext|>", "exact_matche": 0, "edit_similaritie": 0}
# 'input'字段是直接输入qwen2.5-coder模型的内容， 'canonical_solution'是正确答案，
# 'model_gen'是模型生成的内容，'model_gen2'是模型生成的内容（未跳过特殊token）
# 'model_gen_token_ids'是模型生成的token_id
d.keys()

dict_keys(['name', 'language', 'prompt', 'suffix', 'canonical_solution', 'tests', 'input', 'model_gen', 'model_gen_token_ids', 'model_gen2', 'exact_matche', 'edit_similaritie'])

In [8]:
d['input']

'<|fim_prefix|>from typing import List\n\ndef triples_sum_to_zero(l: List[int]) -> bool:\n    """\n    triples_sum_to_zero takes a list of integers as an input.\n    it returns True if there are three distinct elements in the list that\n    sum to zero, and False otherwise.\n\n    >>> triples_sum_to_zero([1, 3, 5, 0])\n    False\n    >>> triples_sum_to_zero([1, 3, -2, 1])\n    True\n    >>> triples_sum_to_zero([1, 2, 3, 7])\n    False\n    >>> triples_sum_to_zero([2, 4, -5, 3, 9, 7])\n    True\n    >>> triples_sum_to_zero([1])\n    False\n    """\n<|fim_suffix|>    l.sort()\n    for i in range(len(l)):\n        for j in range(i + 1, len(l)):\n            for k in range(j + 1, len(l)):\n                if l[i] + l[j] + l[k] == 0:\n                    return True\n    return False\n<|fim_middle|>'

In [9]:
d['model_gen2']

'<|endoftext|>'

In [10]:
output_text

'<|endoftext|>'